# Code notes for BCCNet

source: https://github.com/OlgaIsupova/BCCNet

**utils**: https://github.com/OlgaIsupova/BCCNet/blob/master/utils/utils_dataset_processing.py <br><br>
Two functions: 
1. shuffle_arrays(arrays)
2. shrink_arrays(arrays, shrink_size, is_shuffle=True)

In [1]:
import numpy as np

In [49]:
# assume that our data is in 3d, i.e.for each sample we have 2d input.
# for instance, (6000,28,28) for MNIST
# what the shuffle does is to pick out the rows, and random shuffle those rows

def shuffle_arrays(arrays):
    #if not arrays:
    #    return arrays

    size = arrays[0].shape[0]
    permutation = np.arange(size)
    np.random.shuffle(permutation)
    for i, array in enumerate(arrays):
        arrays[i] = array[permutation]

    return arrays

In [59]:
np.random.seed(100)
x = np.random.choice([0,1], size=(6000, 4, 4))
print(x[0])
print(shuffle_arrays(x)[0])

[[0 0 1 1]
 [1 1 0 0]
 [0 0 0 1]
 [0 0 0 0]]
[[0 0 0 1]
 [0 0 0 0]
 [0 0 1 1]
 [1 1 0 0]]


In [60]:
# what we do is shuffle rows like the previous function
# then output a tuple which contains partial and whole of the example


def shrink_arrays(arrays, shrink_size, is_shuffle=True):
    # if shrink_size in [0.0, 1.0] it specifies fraction of the array size to be extracted, if shrink_size is an
    # integer it specifies the size of the shrunk arrays

    #if not arrays:
    #    return arrays

    if type(shrink_size) == float or type(shrink_size) == np.float64:
        assert(0.0 <= shrink_size <= 1.0)
        size = arrays[0].shape[0]
        shrunk_array_size = int(round(shrink_size * size))
    else:
        shrunk_array_size = shrink_size

    if is_shuffle:
        shuffled_arrays = shuffle_arrays(arrays)
    else:
        shuffled_arrays = arrays

    shrunk_arrays = []
    for array in shuffled_arrays:
        shrunk_arrays.append(array[:shrunk_array_size])

    return shrunk_arrays, shuffled_arrays

In [64]:
np.random.seed(100)
print(x[0])
labelled_train, whole_train = shrink_arrays(x, 0.25) #shrunk_arrays, shuffled_arrays
print(labelled_train[0])
print(whole_train[0])

[[0 0 0 0]
 [1 1 0 0]
 [0 0 1 1]
 [0 0 0 1]]
[[0 0 1 1]]
[[0 0 1 1]
 [1 1 0 0]
 [0 0 0 1]
 [0 0 0 0]]


np function: **numpy.expand_dims()**

In [79]:
np.random.seed(100)
print(x[0])
y = np.expand_dims(x, axis=3)
print(y[0])
print(y.shape)

[[0 0 1 1]
 [1 1 0 0]
 [0 0 0 1]
 [0 0 0 0]]
[[[0]
  [0]
  [1]
  [1]]

 [[1]
  [1]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  [1]]

 [[0]
  [0]
  [0]
  [0]]]
(6000, 4, 4, 1)


In [2]:
n_classes = 2
reliability_level = 0.6
alpha = (reliability_level - (1 - reliability_level) / (n_classes - 1)) * np.eye(n_classes) + \
            (1 - reliability_level) / (n_classes - 1)
print(alpha)

confusion_matrices = np.zeros((2, 2, 5), dtype=np.float64)
for k in np.arange(5):
    for i in np.arange(2):
        confusion_matrices[i, :, k] = np.random.dirichlet(alpha[i, :])
print(confusion_matrices)
print(confusion_matrices.shape)


x = np.cumsum(confusion_matrices[:, :, 1], axis=1)
print(x)



[[0.6 0.4]
 [0.4 0.6]]
[[[0.06048922 0.96905084 0.70739406 0.62198313 0.21330623]
  [0.93951078 0.03094916 0.29260594 0.37801687 0.78669377]]

 [[0.03826223 0.85318721 0.47681111 0.63198369 0.97567575]
  [0.96173777 0.14681279 0.52318889 0.36801631 0.02432425]]]
(2, 2, 5)
[[0.96905084 1.        ]
 [0.85318721 1.        ]]


In [ ]:
# we have cm with shape (2,2,5) named as: confusion_matrices


In [7]:
x = np.random.binomial(1, 0.8, (10, 5))
print(x)

y = np.random.rand(10,1)
print(y)


[[1 1 1 0 1]
 [1 1 1 1 0]
 [1 1 0 1 1]
 [0 1 1 1 1]
 [1 1 1 0 0]
 [0 0 1 1 0]
 [1 1 0 0 1]
 [1 1 1 1 1]
 [1 0 0 1 1]
 [1 1 1 0 1]]
[[0.21465633]
 [0.41011371]
 [0.55321564]
 [0.95173549]
 [0.87471379]
 [0.85582064]
 [0.75820016]
 [0.89383032]
 [0.68797276]
 [0.87556469]]


In [3]:
import scipy.special as ss
def expected_log_Dirichlet_parameters(param):
    size = param.shape
    result = np.zeros_like(param)

    if len(size) == 1:
        result = ss.psi(param) - ss.psi(np.sum(param))
    elif len(size) == 2: # when we take A_0 for everyone
        result = ss.psi(param) - np.transpose(np.tile(ss.psi(np.sum(param, 1)), (size[1], 1)))
    elif len(size) == 3: # most of time for posterior cm
        for i in range(size[2]):
            result[:, :, i] = ss.psi(param[:, :, i]) - \
                              np.transpose(np.tile(ss.psi(np.sum(param[:, :, i], 1)), (size[1], 1)))
    else:
        raise Exception('param can have no more than 3 dimensions')

    return result

In [4]:
Elog = expected_log_Dirichlet_parameters(confusion_matrices)
print(Elog.shape)
print(Elog)

# Elog set up ready to use



(2, 2, 5)
[[[-1.64365426e+01 -5.20937773e-02 -6.22026934e-01 -8.85794555e-01
   -4.38318547e+00]
  [-1.04153344e-01 -3.22612700e+01 -3.01796054e+00 -2.15233632e+00
   -4.18788551e-01]]

 [[-2.60741958e+01 -2.71394344e-01 -1.50546374e+00 -8.51954300e-01
   -4.07389547e-02]
  [-6.47615937e-02 -6.59280359e+00 -1.27619286e+00 -2.23475048e+00
   -4.10719223e+01]]]


In [5]:
import random
x = np.arange(100)
for i, j in enumerate(x):
    x[i] = 1

y = np.arange(100)
for i, j in enumerate(y):
    y[i] = 0

z = np.arange(700)
for i, j in enumerate(z):
    z[i] = -1


p = np.append(x,y)
q = np.append(p,z)

q = q.reshape((3,60,5))
#I=3, U=60(20x3), K=5

np.random.seed(8)
np.random.shuffle(q) # the input X, the labels from 5 volunteers towards 3 images, with total 60 anchor boxes 

I, U, K = q.shape
print(q.shape)



(3, 60, 5)


In [6]:
for k in range(K):
        inds = np.where(q[:,:, k] > -1) #rule out missing values
        rho[inds[0],inds[1], :] = rho[inds[0],inds[1], :] + np.transpose(Elog[:, np.squeeze(q[inds[0],inds[1], k]), k])
print(rho.shape)


NameError: name 'rho' is not defined

In [60]:
# rho = rho - np.transpose(np.tile(np.max(rho, 1), (J, 1)))

check1 = np.arange(24).reshape(2,3,4) #i.e. I=2, U=3, M=4
print(check1)
print(np.max(check1, 2).shape)
print(np.max(check1, 2))
print(np.tile(np.transpose(np.max(check1, 2)), (4, 1, 1)).shape)
print(np.tile(np.transpose(np.max(check1, 2)), (4, 1, 1)))
check1 = check1 - np.transpose(np.tile(np.transpose(np.max(check1, 2)), (4, 1, 1)))
print(check1)

print(np.transpose(np.tile(np.sum(np.exp(check1), 2), (4, 1, 1))).shape)



[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
(2, 3)
[[ 3  7 11]
 [15 19 23]]
(4, 3, 2)
[[[ 3 15]
  [ 7 19]
  [11 23]]

 [[ 3 15]
  [ 7 19]
  [11 23]]

 [[ 3 15]
  [ 7 19]
  [11 23]]

 [[ 3 15]
  [ 7 19]
  [11 23]]]
[[[-3 -2 -1  0]
  [-3 -2 -1  0]
  [-3 -2 -1  0]]

 [[-3 -2 -1  0]
  [-3 -2 -1  0]
  [-3 -2 -1  0]]]
(3, 2, 4)


In [67]:
print(np.sum(np.exp(check1), 2).shape)
print(np.transpose(np.tile(np.transpose(np.sum(np.exp(check1), 2)),(4, 1, 1))).shape)

(2, 3)
(2, 3, 4)


In [64]:
check2 = np.arange(8).reshape(2,4) #i.e. I=2, M=4
#print(check2)
#print(np.max(check2, 1).shape)
#print(np.max(check2, 1))
#print(np.tile(np.max(check2, 1), (4, 1)).shape)
#print(np.tile(np.max(check2, 1), (4, 1)))
#print(np.transpose(np.tile(np.max(check2, 1), (4, 1))).shape)

print(np.sum(np.exp(check2), 1))

print(np.transpose(np.tile(np.sum(np.exp(check2), 1), (4, 1))))
print(np.transpose(np.tile(np.sum(np.exp(check2), 1), (4, 1))).shape)

[  31.19287485 1703.07326106]
[[  31.19287485   31.19287485   31.19287485   31.19287485]
 [1703.07326106 1703.07326106 1703.07326106 1703.07326106]]
(2, 4)


In [8]:
x = np.array([[-1, 0, 1], [1, 0, 1],[1, 1, 1]]) # the answer from 3 indiduals, I = 3, W = 3
rho = np.zeros((3,2)) # nn_output, are all zeros, in dim I x M
for i in range(x.shape[1]):
    inds = np.where(x[:, i] > -1)
    print(inds)
    print(rho[inds, :])
    print(np.transpose(Elog[:, np.squeeze(x[inds, i]), i]))
    #rho[inds, :] = rho[inds, :] + np.transpose(Elog[:, np.squeeze(x[inds, i]), i])
print(rho)   

(array([1, 2]),)
[[[0. 0.]
  [0. 0.]]]
[[-0.10415334 -0.06476159]
 [-0.10415334 -0.06476159]]
(array([0, 1, 2]),)
[[[0. 0.]
  [0. 0.]
  [0. 0.]]]
[[ -0.05209378  -0.27139434]
 [ -0.05209378  -0.27139434]
 [-32.26127003  -6.59280359]]
(array([0, 1, 2]),)
[[[0. 0.]
  [0. 0.]
  [0. 0.]]]
[[-3.01796054 -1.27619286]
 [-3.01796054 -1.27619286]
 [-3.01796054 -1.27619286]]
[[0. 0.]
 [0. 0.]
 [0. 0.]]


In [9]:
# x = np.array([[-1, 0, 1], [1, 0, 1],[1, 1, 1]]) # the answer from 3 indiduals, #I=3, U=60(20x3), K=5
# here we have M X N X K
rho = rho - np.transpose(np.tile(np.transpose(np.max(rho, 2)), (2, 1, 1)))
q_t = np.exp(rho) / np.maximum(1e-60, np.transpose(np.tile(np.transpose(np.sum(np.exp(rho), 2)), (2, 1, 1))))
q_t = np.maximum(1e-60, q_t)
print(q_t.shape)

M = 2
N = 2
K = 5
F_iu = np.zeros((M, N, K), dtype=np.float64)
for k in range(K):
    for n in range(N):
        inds0 = np.where(q[:, :, k] == n)[0]
        inds1 = np.where(q[:, :, k] == n)[1]
        F_iu[:, n, k] = np.sum(q_t[inds0, inds1, :], 0)
print(F_iu.shape)
print(F_iu)


AxisError: axis 2 is out of bounds for array of dimension 2

In [137]:
alpha0 = np.array([[0.4, 0.6],[0.6, 0.4]])
alpha0 = np.transpose(np.tile(alpha0, (5,1,1)))
print(alpha0.shape)



(2, 2, 5)


In [141]:
def logB_from_Dirichlet_parameters(alpha): # used to compute loss. ???
    logB = np.sum(ss.gammaln(alpha)) - ss.gammaln(np.sum(alpha))

    return logB

In [142]:
def compute_lower_bound_likelihood(alpha0_volunteers, alpha_volunteers, q_t, rho, nn_output):
    K = alpha0_volunteers.shape[2]

    ll_pi_worker = 0
    for k in range(K):
        ll_pi_worker = ll_pi_worker - np.sum(logB_from_Dirichlet_parameters(alpha0_volunteers[:, :, k]) -
                                             logB_from_Dirichlet_parameters(alpha_volunteers[:, :, k]))

    ll_t = -np.sum(q_t * rho) + np.sum(np.log(np.sum(np.exp(rho), axis=2)))

    ll_nn = np.sum(q_t * nn_output) - np.sum(np.log(np.sum(np.exp(nn_output), axis=2)))

    ll = ll_pi_worker + ll_t + ll_nn  # VB lower bound

    return ll

In [10]:
alpha0 #225
alpha1 = alpha0
q_tt = np.arange(360).reshape(3, 60, 2)
rho = np.arange(360).reshape(3, 60, 2)
nn_output = np.random.rand(3, 60, 2)


NameError: name 'alpha0' is not defined

In [145]:
checkll = compute_lower_bound_likelihood(alpha0, alpha0, q_tt, rho, nn_output)

In [146]:
checkll

-15420699.672973178